In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_parsed_html(url: str):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return None

test_url = "https://www.transfermarkt.co.uk/raphinha/profil/spieler/411295"

soup = get_parsed_html(test_url)

if soup:
    market_value_wrapper = soup.find('a', class_='data-header__market-value-wrapper')

    if market_value_wrapper:
        market_value_parts = [text.strip() for text in market_value_wrapper.stripped_strings if "Last update" not in text]
        market_value = ''.join(market_value_parts)

        last_update_tag = market_value_wrapper.find('p', class_='data-header__last-update')
        last_update = last_update_tag.get_text(strip=True) if last_update_tag else "No update information available"

        print(f"Market Value: {market_value}")
        print(f"Last Update: {last_update}")
    else:
        print("Market value wrapper not found.")
else:
    print("Failed to fetch and parse the webpage.")

Market Value: €60.00m
Last Update: Last update: Oct 11, 2024


In [14]:
market_value

'€60.00m'